In [4]:
import numpy as np 
import pandas as pd
import cv2
import os
from tqdm import tqdm
from skimage.feature import graycomatrix, graycoprops

In [5]:
def compute_glcm_descriptor(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    
    # Configuration 
    distance = [3]  
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  
    properties = ['contrast', 'correlation', 'energy', 'homogeneity']  
    
    glcm = graycomatrix(image, distances=distance, angles=angles, symmetric=True, normed=True)
    
    features = []
    for prop in properties:
        feature = graycoprops(glcm, prop).flatten()
        features.extend(feature)
    
    return np.array(features)

In [7]:
def process_directory(base_path):
    all_features = []
    all_labels = []
    image_paths = []

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                label = 0 if 'Negative' in root else 1
                image_paths.append((os.path.join(root, file), label))
    
    for image_path, label in tqdm(image_paths, desc="Processing"):
        features = compute_glcm_descriptor(image_path)
        if features is not None:
            all_features.append(features)
            all_labels.append(label)
    
    df = pd.DataFrame(all_features)
    df['label'] = all_labels
    
    return df

In [19]:
base_path = r'D:\Rice_photos\Rice_photos\Xi23'
df = process_directory(base_path)

Processing:   0%|          | 0/4145 [00:00<?, ?it/s]

Processing: 100%|██████████| 4145/4145 [01:47<00:00, 38.72it/s]


In [20]:
df.to_csv('glcm_features_Xi23.csv', index=False)